In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# data_path = os.path.expanduser('~')+'/.mozilla/firefox/jgpzezfu.default'
# files = os.listdir(data_path)
# history_db = os.path.join(data_path, 'places.sqlite')
history_db = 'dataset/places.sqlite'
c = sqlite3.connect(history_db)
cursor = c.cursor()


In [3]:
sql_command = """
SELECT datetime(hv.visit_date/1000000,'unixepoch') as date, 
h.url as url, h.visit_count as vc
FROM moz_places h, moz_historyvisits hv
WHERE h.id = hv.place_id 
and date > '2018-30-12'
order by date
"""
cursor.execute(sql_command)
results = cursor.fetchall()


In [4]:
df = pd.DataFrame(results, columns=['date','url','count'])
df['date'] = pd.to_datetime(df.date)
df.head(10)

,date,url,count
0,2019-01-02 05:19:29,file:///run/user/1001/jupyter/nbserver-3729-op...,1
1,2019-01-02 05:19:31,http://localhost:8888/tree?token=bbfd697d6e63e...,1
2,2019-01-02 05:19:32,http://localhost:8888/tree,235
3,2019-01-02 05:22:15,http://localhost:8888/tree/kaggle_data_science,128
4,2019-01-02 05:23:24,http://localhost:8888/notebooks/kaggle_data_sc...,2
5,2019-01-02 05:23:28,http://localhost:8888/notebooks/kaggle_data_sc...,4
6,2019-01-02 05:23:30,http://localhost:8888/notebooks/kaggle_data_sc...,1
7,2019-01-02 05:23:33,http://localhost:8888/edit/kaggle_data_science...,2
8,2019-01-02 05:24:28,http://localhost:8888/notebooks/kaggle_data_sc...,1
9,2019-01-02 05:25:07,http://localhost:8888/tree/kaggle_data_science...,1


In [5]:
from urllib.parse import urlparse
urlparse('https://xgboost.readthedocs.io/en/latest/python/python_intro.html#prediction')

ParseResult(scheme='https', netloc='xgboost.readthedocs.io', path='/en/latest/python/python_intro.html', params='', query='', fragment='prediction')

In [6]:
df['url_scheme']   = df['url'].apply(lambda x: urlparse(x).scheme)
df['url_netloc']   = df['url'].apply(lambda x: urlparse(x).netloc)
df['url_path']     = df['url'].apply(lambda x: urlparse(x).path)
df['url_fragment'] = df['url'].apply(lambda x: urlparse(x).fragment)
df['url_query']    = df['url'].apply(lambda x: urlparse(x).query)
df.head()

,date,url,count,url_scheme,url_netloc,url_path,url_fragment,url_query
0,2019-01-02 05:19:29,file:///run/user/1001/jupyter/nbserver-3729-op...,1,file,,/run/user/1001/jupyter/nbserver-3729-open.html,,
1,2019-01-02 05:19:31,http://localhost:8888/tree?token=bbfd697d6e63e...,1,http,localhost:8888,/tree,,token=bbfd697d6e63ecc46211932cb5c945192c2726a1...
2,2019-01-02 05:19:32,http://localhost:8888/tree,235,http,localhost:8888,/tree,,
3,2019-01-02 05:22:15,http://localhost:8888/tree/kaggle_data_science,128,http,localhost:8888,/tree/kaggle_data_science,,
4,2019-01-02 05:23:24,http://localhost:8888/notebooks/kaggle_data_sc...,2,http,localhost:8888,/notebooks/kaggle_data_science/keras%20lstm%20...,,


In [7]:
df.isnull().sum()

date            0
url             0
count           0
url_scheme      0
url_netloc      0
url_path        0
url_fragment    0
url_query       0
dtype: int64

In [8]:
df.groupby('url_netloc')['count'].sum().sort_values(ascending=False).head(50)

url_netloc
www.goal.com                    2761202
www.google.com                  1343953
www.youtube.com                  392635
nwm.iitk.ac.in                   212232
mail.google.com                  182839
localhost:8888                    54039
www.kaggle.com                    32224
ironport2.iitk.ac.in              21345
github.com                        20475
www.facebook.com                  18430
dare2compete.com                   9170
stackoverflow.com                  7404
scikit-learn.org                   3852
localhost:9090                     3445
localhost:9099                     3320
localhost:8800                     3251
www.overleaf.com                   3089
localhost:9999                     2815
datahack.analyticsvidhya.com       2640
localhost:9000                     2534
www.hackerearth.com                2260
docs.google.com                    2238
keras.io                           2141
localhost:8889                     2079
pingala.iitk.ac.in           

In [ ]:
# moz_places

# The moz_places table holds some of the information necessary to reconstruct the browser history.

#     id INTEGER PRIMARY KEY
#     url LONGVARCHAR (The whole URL string)
#     title LONGVARCHAR (The title presented from the TITLE tags on the page)
#     rev_host LONGVARCHAR (this is the host name from the URL in reverse)
#     visit_count INTEGER
#     hidden INTEGER
#     typed INTEGER
#     favicon_id INTEGER
#     frecency INTEGER

# moz_historyvisits

# The moz_historyvisits table holds the other information that you need to link up with moz_places to reconstruct the browser history.

#     id INTEGER
#     from_visit INTEGER
#     place_id INTEGER
#     visit_date INTEGER 
#     visit_type INTEGER
#     session INTEGER

# The place_id column of the moz_historyvisits table corresponds to the id column of the moz_places table.

# The visit_date column keeps time in PRTime format, a 64-bit integer representing the number of microseconds since midnight (00:00:00) 1 January 1970 Coordinated Universal Time (UTC). This level of precision may be more than what is required for a forensic application, but the PRTime format can easily be converted into CTime format by dividing by 1,000,000. The datetime function in SQLite can be used to convert CTime to human readable format, as demonstrated in the example below.

# Gathering browser history
# Live browser history for Firefox 3 can be gathered by connecting to the Sqlite database and performing simple queries. For example, the URLs visited and the date and time of the visit can be gathered with this query:

# SELECT datetime(histv.visit_date/1000000,'unixepoch'), hist.url, hist.visit_count
# FROM moz_places hist, moz_historyvisits histv
# WHERE hist.id = histv.place_id